In [1]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
from sentence_transformers import SentenceTransformer
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from pyvis.network import Network


/home/arunav/.conda/envs/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-08 21:15:15.720069: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-08 21:15:15.750217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-08 21:15:16.305383: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warni

In [2]:
text = "In a tiny village, a curious girl named Anaya discovered a magical, titleless book in the old library. As she read, she was transported into a world of enchanted forests and talking animals, meeting a brave explorer named Aiden. Together, they embarked on thrilling adventures, solving ancient riddles and rescuing kingdoms. When it was time to return, Aiden reminded Anaya that the magic was always within her. Back in the library, Anaya shared her tales with the villagers, inspiring them to dream and explore, keeping the magic alive in their hearts."

In [3]:

# Step 1: Text Summarization with TensorFlow
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
inputs = tokenizer.encode("summarize: " + text, return_tensors="tf", max_length=512, truncation=True)
with tf.device('/CPU:0'):
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:", summary)

# Step 2: Key Sentence Extraction
def extract_key_sentences(text, num_sentences=5):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    sentences = text.split('. ')
    embeddings = model.encode(sentences)
    similarities = cosine_similarity(np.array(embeddings))
    sentence_scores = similarities.sum(axis=1)
    top_sentence_indices = sentence_scores.argsort()[-num_sentences:][::-1]
    key_sentences = [sentences[i] for i in top_sentence_indices]
    return key_sentences

key_sentences = extract_key_sentences(summary)
print("Key Sentences:", key_sentences)

# Step 3: Dependency Parsing and Hierarchical Structuring
nlp = spacy.load('en_core_web_sm')
parsed_sentences = [nlp(sentence) for sentence in key_sentences]
relationships = []
for sent in parsed_sentences:
    for token in sent:
        if token.dep_ in ("nsubj", "dobj", "iobj", "pobj", "attr"):
            relationships.append((token.head.text, token.text, sent.text))
print("Relationships:", relationships)

# Step 4: Concept Mapping with Pyvis
net = Network(notebook=True, directed=True)
for rel in relationships:
    net.add_node(rel[0], label=rel[0])
    net.add_node(rel[1], label=rel[1])
    net.add_edge(rel[0], rel[1], title=rel[2])
net.show("flow_chart.html")


2024-07-08 21:15:19.215636: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-08 21:15:19.220261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-08 21:15:19.223572: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Summary: a girl named Anaya discovered a book in the old library in a tiny village. she was transported into a world of enchanted forests and talking animals. they embarked on thrilling adventures, solving ancient riddles and rescuing kingdoms.
Key Sentences: ['she was transported into a world of enchanted forests and talking animals', 'a girl named Anaya discovered a book in the old library in a tiny village', 'they embarked on thrilling adventures, solving ancient riddles and rescuing kingdoms.']
Relationships: [('into', 'world', 'she was transported into a world of enchanted forests and talking animals'), ('of', 'forests', 'she was transported into a world of enchanted forests and talking animals'), ('discovered', 'girl', 'a girl named Anaya discovered a book in the old library in a tiny village'), ('discovered', 'book', 'a girl named Anaya discovered a book in the old library in a tiny village'), ('in', 'library', 'a girl named Anaya discovered a book in the old library in a tiny v